# DAPPER

Thanks to the [team](https://www.nersc.no/about)
* A starting point point for Data Assimilation https://www.nersc.no/group/data-assimilation

In [16]:
%%bash
ECF_PORT=2500 ecflow_start.sh -p 2500 >/dev/null

In [17]:
######################
# from __future__ import print_function
import os, sys
# lib = "/usr/local/apps/ecflow/current/lib/python2.7/site-packages/ecflow"
# lib = "/usr/local/apps/ecflow/current/lib/python3.5/site-packages/ecflow"
lib = "/usr/local/lib/python3.5/site-packages/ecflow"
sys.path.append(lib)
import ecf; from ecf import (Client, Defs, Suite, Family, Task, Defstatus, Edit, Label, Trigger)
try: x = Edit(test="value")  # Edit is present in recent ecf.py module
except:
    class Edit(Variables): pass
home = os.getenv("HOME") + "/ecflow_server"
user = os.getenv("USER")

# SUITE
node = Suite("DA").add(
    Defstatus("suspended"),
    Edit(ECF_HOME=home, ECF_INCLUDE=home + "/include", ECF_FILES=home + "/files",
         ECF_EXTN=".ecg",  # current convention for generated task template extension
         ECF_JOB_CMD="%ECF_JOB% > %ECF_JOBOUT% 2>&1",  # localhost run
         ECF_URL_CMD="firefox %URL%",
         URL="https://www.nersc.no/group/data-assimilation", ),
    Family("make").add(
        Family("get").add(Task("cmd").add(
            Edit(CMD="[ ! -d DAPPER ] && " +
                 "git clone https://github.com/nansencenter/DAPPER.git",
                 ARGS=""))),
        Family("compile").add(
            Trigger(["get"]),
            Task("cmd").add(
                Edit(CMD="cd DAPPER; xterm -T 'python3 example_1.py'")), ), ),
    Family("main").add(Task("cmd").add(
        Label("info", ""),
        Edit(CMD="ecflow_client --label info", ARGS="YOUR PART"), ), ))
# print(node)

# TASK TEMPLATE
fname = home + "/files/cmd.ecg"
if not os.path.isfile(fname):
    with open(fname, 'w') as task_template: print("""#!/bin/bash
%include <head.h>
 %CMD:echo% %ARGS:%
%include <tail.h>""", file=task_template)

# DEFS
defs = Defs()
defs.add_suite(node)
path = '/' + node.name()
# print(defs)

# CLIENT
client = Client("localhost@%s" % os.getenv("ECF_PORT", 2500))  # PYTHON CLIENT
if node.name() not in client.suites():
  client.load(defs)  # load/replace the top node (suite)
  client.begin_suite(node.name())  # BEGIN suite: UNKNOWN -> QUEUED
else: client.replace(path, defs);  # print("# REPLACE " + path, client)  # load/replace the top node (suite)
# client.resume(path)  # RESUME suite: SUSPENDED -> create job and submit

In [18]:
import os
os.system("ecflow_ui &")

0